# Euclid Q1 Lenses

* **Phil Marshall, Phil Holloway, Ralf Kaehler, Ferro Shao**
* DP1
* data.lsst.cloud
* r29.1.1
* Fri July 11 2025

## Goals

* Extract _ugrizy_ coadd image cutouts for each Euclid Q1 strong lens candidate in the ECDFS and EDFS DP1 fields
* Visualize them as _gri_ color composites.
* Stretch: deconvolve them using the Rubin SharPy by Kaehler et al (in prep)

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afw_display
import lsst.geom as geom
from lsst.afw.image import MultibandExposure
import numpy as np
from astropy.visualization import make_lupton_rgb
import matplotlib.pyplot as plt

afw_display.setDefaultBackend('matplotlib')

## Cutout Image Extraction

First we need to make a list (or better, a table) of targets. Then, for each one, we find out which DP1 coadd patch it lies in. (We'll need to choose which patch, for systems that lie in the patch overlap regions and hence in multiple patches.) Then, we loop over patches and bands, uploading a patch image and extracting all the cutouts we can - which will mean getting the image coordinates for each system

In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")
assert butler is not None

In [ ]:
butler.get_dataset_type('deep_coadd').dimensions.required

### Single Sky Position, Single Band

Let's just try extracting a single 32x32 pixel cutout image in one band.

In [ ]:
ra = 59.626134
dec = -49.06175

band = 'i'

Turn the coordinates into an IAU standard object name, we'll need this later:

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

# Example RA and Dec coordinates
rahms = ra * u.hourangle  # 12 hours, 30 minutes, 36 seconds
decdms = dec * u.deg    # +12 degrees, 24 minutes, 0 seconds

# Create a SkyCoord object
coordinates = SkyCoord(ra=rahms, dec=decdms, frame='icrs')

# Format the coordinates into an IAU-style string
name = (f'EUCLID J{coordinates.ra.to_string(unit=u.hourangle, sep="", precision=1, pad=True)}'
                  f'{coordinates.dec.to_string(sep="", precision=0, alwayssign=True, pad=True)}') #

print(name)

We need to find the tract and patch that this target is in. This approach was adopted from the CST Tutorial ["03a Image Display and Manipulation"](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/03a_Image_Display_and_Manipulation.ipynb).

In [ ]:
radec = geom.SpherePoint(ra, dec, geom.degrees)
cutoutSize = geom.ExtentI(32, 32)

skymap = butler.get("skyMap")
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)

patch = tractInfo.getSequentialPatchIndex(patchInfo)
tract = tractInfo.getId()

dataId = {'tract': tract, 'patch': patch, 'band': band}

When testing, it can be useful to upload the whole patch image and inspect it.

In [ ]:
# deep_coadd = butler.get('deep_coadd', band=band, tract=tract, patch=patch)
# coadd

In [ ]:
# fig = plt.figure(figsize=(6,6))
# display = afw_display.Display(frame=fig)
# display.scale('asinh', 'zscale')
# display.mtv(coadd.image)
# plt.show()

Now to define a small bounding box, and extract the pixels in it. This first cell below _should_ work, but doesn't - maybe some tract/patch confusion. There could be some speed up here at some point, making multiple cutouts from the same patch image using the calexp object's native factory method.

In [ ]:
# xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
# bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

# cutout = coadd.Factory(coadd, bbox)

Here's some code that does work: define the bounding box, then just grab that part of the image.

In [ ]:
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

parameters = {'bbox': bbox}

cutout = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

Quick look to check we got our object!

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
plt.show()

### Single Object, Multiple Bands

Loop over all 6 bands and extract the cutout image in each one.

In [ ]:
bands = ["u","g","r","i","z","y"]
cutout = {}

for band in bands:
    dataId = {'tract': tract, 'patch': patch, 'band': band}
    cutout[band] = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

In [ ]:
cutout

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout["g"].image)
plt.show()

OK - we have 6 cutouts for this target, so can go ahead and make a color composite - see below for this. It took about 5 secs to make them all: we'll need to keep an eye on this, and return to the `factory` approach to try and speed things up a bit.

## _gri_ Composite Image Visualization

Here's a useful function, adapted from the CST Tutorial ["03a Image Display and Manipulation"](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/03a_Image_Display_and_Manipulation.ipynb).

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=100, Q=1, name=None):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    plt.axis("off")
    ax.imshow(rgb, interpolation='nearest', origin='lower')

    if name is not None:
        plt.text(0, 31, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.text(0, 2, bgr, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')

First we need to package our cutouts into a MultibandExposure object, then we pass that to the RGB composite generation function.

In [ ]:
cutouts = [cutout[band] for band in bands]
multibandexposure = MultibandExposure.fromExposures(bands, cutouts)

In [ ]:
showRGB(multibandexposure.image, bgr='gri', figsize=(3,3), stretch=60, Q=1, name=name)

Choosing the stretch and Q can be a bit fiddly - this is best done when visualizing the whole set of cutouts in a gallery. This is what we will do next.

## Appendix

The code below is from the Cutout Factory demo notebook by Melissa Graham at https://github.com/lsst/cst-dev/blob/main/MLG_sandbox/random/cutout_factory_demo_2025-06-05.ipynb, and is experimented with in this notebook further up.

In [ ]:


import lsst.afw.display as afw_display
from lsst.daf.butler import Butler
import lsst.geom as geom
import matplotlib.pyplot as plt

afw_display.setDefaultBackend('matplotlib')

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
dataId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', **dataId)

In [ ]:
fig = plt.figure(figsize=(3,3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(calexp.image)
plt.show()

In [ ]:
cutoutSize = geom.ExtentI(301, 301)

xy1 = geom.PointI(2250, 700)
bbox1 = geom.BoxI(xy1 - cutoutSize // 2, cutoutSize)

xy2 = geom.PointI(400, 1750)
bbox2 = geom.BoxI(xy2 - cutoutSize // 2, cutoutSize)

In [ ]:
cutout1 = calexp.Factory(calexp, bbox1)
cutout2 = calexp.Factory(calexp, bbox2)

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout1.image)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout2.image)
plt.show()